# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
quantities = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity' : 'sum'})
quantities

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
table = pd.pivot_table(quantities, values='Quantity', index='ProductName', columns='CustomerID')
table = table.fillna(0)
table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
from scipy.spatial.distance import pdist, squareform

squareform(pdist(table, 'euclidean'))

array([[  0.        , 279.99642855, 253.66907577, ..., 255.67166444,
        258.0910692 , 278.47800631],
       [279.99642855,   0.        , 271.49953959, ..., 265.7367118 ,
        279.55500353, 286.80655502],
       [253.66907577, 271.49953959,   0.        , ..., 245.27535547,
        253.01580978, 269.43644891],
       ...,
       [255.67166444, 265.7367118 , 245.27535547, ...,   0.        ,
        264.14200726, 268.28343221],
       [258.0910692 , 279.55500353, 253.01580978, ..., 264.14200726,
          0.        , 273.15014186],
       [278.47800631, 286.80655502, 269.43644891, ..., 268.28343221,
        273.15014186,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), index=table.columns, columns=table.columns)
distances
similarities = dict(distances[669].sort_values(ascending=False)[1:6])
similarities.keys()

dict_keys([3535, 3317, 639, 1008, 3305])

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
new = quantities.loc[[*similarities.keys()], :]
new

Quantity
CustomerID ProductName                              
639        Anchovy Paste - 56 G Tube               1
           Appetizer - Mushroom Tart               1
           Assorted Desserts                       1
           Bread - Italian Roll With Herbs         1
           Bread - Raisin Walnut Oval              1
...                                              ...
3535       Tray - 16in Rnd Blk                     1
           Wanton Wrap                             1
           Wine - Blue Nun Qualitatswein           1
           Wine - Magnotta, Merlot Sr Vqa          1
           Yoghurt Tubes                           1

[278 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
ranked = new.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False)
ranked

,Quantity
ProductName,
"Veal - Inside, Choice",4
Sardines,3
"Sauce - Gravy, Au Jus, Mix",3
Wine - Ej Gallo Sierra Valley,3
Cheese - Cambozola,3
...,...
Garlic,1
Garlic - Elephant,1
"Garlic - Primerba, Paste",1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
mergeado = ranked.merge(table[669], on = 'ProductName')
notbuy = mergeado.loc[mergeado[669] == 0]
notbuy.head()

,Quantity,669
ProductName,,
"Veal - Inside, Choice",4,0.0
Sardines,3,0.0
Wine - Ej Gallo Sierra Valley,3,0.0
"Pepper - Black, Whole",3,0.0
Wine - Blue Nun Qualitatswein,3,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [15]:
lst=[]
for cust in data['CustomerID']:
    lst.append(cust)
seteado=list(set(lst))


In [23]:
empty = {}

for k in seteado:
    similar=distances[k].sort_values(ascending=False)[1:6]
    new1 = quantities.loc[list(similar.index)]
    ranked1 = new1.reset_index().groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False)
    mergeado1 =  ranked1.merge(table[k], on = 'ProductName')
    filtered = mergeado1.loc[mergeado1[k] == 0][1:6].sort_values(by='Quantity',ascending=False)
    listando = list(filtered.index)
    empty[k] = listando
    
empty

{83973: ['Veal - Slab Bacon',
  'Bandage - Fexible 1x3',
  'Ezy Change Mophandle',
  'Chicken - Wieners',
  'Puree - Passion Fruit'],
 59399: ['Knife Plastic - White',
  'Towels - Paper / Kraft',
  'Nantucket - Pomegranate Pear',
  'Snapple - Iced Tea Peach',
  'Cookie Chocolate Chip With'],
 92168: ['Veal - Brisket, Provimi,bnls',
  'Snapple Lemon Tea',
  'Spinach - Baby',
  'Pop Shoppe Cream Soda',
  'Bread - Calabrese Baguette'],
 49159: ['Cookies - Assorted',
  'Puree - Mocha',
  'Cheese - Parmesan Grated',
  'Cheese - Victor Et Berthold',
  'Chicken - Leg, Boneless'],
 18441: ['Anchovy Paste - 56 G Tube',
  'Sprouts - Baby Pea Tendrils',
  'Lemonade - Natural, 591 Ml',
  'Sausage - Breakfast',
  'Soup Knorr Chili With Beans'],
 22536: ['Cheese - Mozzarella',
  'Cheese Cloth No 100',
  'Bread - Italian Roll With Herbs',
  'Bread - Hot Dog Buns',
  'Table Cloth - 53x69 Colour'],
 86028: ['Pork - Hock And Feet Attached',
  'Vinegar - Tarragon',
  'Bread Foccacia Whole',
  'Lettuce - 

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [27]:
result = pd.DataFrame(empty).T
result

,0,1,2,3,4
83973,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners,Puree - Passion Fruit
59399,Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach,Cookie Chocolate Chip With
92168,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda,Bread - Calabrese Baguette
49159,Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold,"Chicken - Leg, Boneless"
18441,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast,Soup Knorr Chili With Beans
...,...,...,...,...,...
77814,Cake - Cake Sheet Macaroon,Sea Bass - Whole,Squid U5 - Thailand,Guinea Fowl,Macaroons - Two Bite Choc
32753,Sherry - Dry,"Chestnuts - Whole,canned",Napkin White - Starched,Sprouts - Alfalfa,Appetizer - Sausage Rolls
20476,"Pasta - Detalini, White, Fresh","Cheese - Boursin, Garlic / Herbs",Beans - Kidney White,Wine - Blue Nun Qualitatswein,Lime Cordial - Roses
67582,Ecolab - Solid Fusion,Wine - Redchard Merritt,Mussels - Frozen,Cheese - Bocconcini,"Peas - Pigeon, Dry"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [29]:
distances2 = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'jaccard'))), index=table.columns, columns=table.columns)
empty2={}
for k in seteado:
    similar=distances2[k].sort_values(ascending=False)[1:6]
    new1 = quantities.loc[list(similar.index)]
    ranked1 = new1.reset_index().groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False)
    mergeado1 =  ranked1.merge(table[k], on = 'ProductName')
    filtered = mergeado1.loc[mergeado1[k] == 0][1:6].sort_values(by='Quantity',ascending=False)
    listando = list(filtered.index)
    empty2[k] = listando

result2 = pd.DataFrame(empty2).T

In [30]:
result2

,0,1,2,3,4
83973,Milk - 1%,"Chocolate - Semi Sweet, Calets",Pork - Belly Fresh,Bread Crumbs - Japanese Style,Bread - Rye
59399,"Cheese - Brie, Triple Creme","Rum - Coconut, Malibu",Cheese - Parmesan Cubes,Gloves - Goldtouch Disposable,Wine - Hardys Bankside Shiraz
92168,Wiberg Super Cure,Salmon - Sockeye Raw,Cumin - Whole,Scallops - 10/20,Tea - English Breakfast
49159,Cookie Dough - Double,Mussels - Cultivated,Cheese - Bocconcini,Macaroons - Two Bite Choc,Soup - Campbells Bean Medley
18441,Macaroons - Two Bite Choc,Bread - English Muffin,"Juice - Cranberry, 341 Ml","Beef - Ground, Extra Lean, Fresh",Nantucket - Pomegranate Pear
...,...,...,...,...,...
77814,Cumin - Whole,Beef - Montreal Smoked Brisket,Soupcontfoam16oz 116con,Squid - Tubes / Tenticles 10/20,"Lemonade - Natural, 591 Ml"
32753,"Liners - Banana, Paper",Whmis - Spray Bottle Trigger,"Tart Shells - Sweet, 4","Lentils - Red, Dry",Table Cloth 62x114 White
20476,Broom - Corn,Mustard - Seed,"Cheese - Brie,danish",Mayonnaise - Individual Pkg,Chef Hat 20cm
67582,"Lamb - Pieces, Diced",Pie Filling - Cherry,"Beer - Alexander Kieths, Pale Ale",Rambutan,Sponge Cake Mix - Chocolate
